In [1]:
!ls

sample_data


In [2]:
# ===========================
# STEP 1: SETUP & IMPORTS
# ===========================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

print("TensorFlow version:", tf.__version__)

# ===========================
# MOUNT GOOGLE DRIVE
# ===========================
from google.colab import drive
drive.mount('/content/drive')


TensorFlow version: 2.19.0
Mounted at /content/drive


In [3]:
# ===========================
# PROJECT DIRECTORIES
# ===========================
PROJECT_NAME = "MNIST_DCGAN"
BASE_DIR = f"/content/drive/MyDrive/{PROJECT_NAME}"

CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
IMAGE_DIR = os.path.join(BASE_DIR, "generated_images")
MODEL_DIR = os.path.join(BASE_DIR, "models")
PLOT_DIR = os.path.join(BASE_DIR, "plots")

for folder in [BASE_DIR, CHECKPOINT_DIR, IMAGE_DIR, MODEL_DIR, PLOT_DIR]:
    os.makedirs(folder, exist_ok=True)

print("All folders created successfully")


All folders created successfully


In [4]:
# ===========================
# HYPERPARAMETERS
# ===========================
EPOCHS = 150          # More epochs = better quality
BATCH_SIZE = 256
LATENT_DIM = 100
SAVE_INTERVAL = 10

GEN_LR = 0.0002
DISC_LR = 0.0002
BETA_1 = 0.5

tf.random.set_seed(42)
np.random.seed(42)


In [5]:
# ===========================
# LOAD MNIST DATASET
# ===========================
(train_images, _), (_, _) = keras.datasets.mnist.load_data()

# Reshape and normalize (-1 to 1)
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32")
train_images = (train_images - 127.5) / 127.5

dataset = tf.data.Dataset.from_tensor_slices(train_images)
dataset = dataset.shuffle(60000).batch(BATCH_SIZE, drop_remainder=True)

print("MNIST dataset loaded")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MNIST dataset loaded


In [6]:
# ===========================
# GENERATOR MODEL
# ===========================
def build_generator():
    model = keras.Sequential([
        layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((7, 7, 256)),

        layers.Conv2DTranspose(128, 5, strides=1, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(1, 5, strides=2, padding="same",
                               use_bias=False, activation="tanh")
    ])
    return model

generator = build_generator()
generator.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12544)          │     1,254,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12544)          │        50,176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 7, 7, 128)      │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 14, 14, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 28, 28, 1)      │         1,600 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,330,944 (8.89 MB)

 Trainable params: 2,305,472 (8.79 MB)

 Non-trainable params: 25,472 (99.50 KB)

In [7]:
# ===========================
# DISCRIMINATOR MODEL
# ===========================
def build_discriminator():
    model = keras.Sequential([
        layers.Conv2D(64, 5, strides=2, padding="same",
                      input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, 5, strides=2, padding="same"),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

discriminator = build_discriminator()
discriminator.summary()



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# ===========================
# LOSS & OPTIMIZERS
# ===========================
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

generator_optimizer = keras.optimizers.Adam(GEN_LR, beta_1=BETA_1)
discriminator_optimizer = keras.optimizers.Adam(DISC_LR, beta_1=BETA_1)


In [9]:
# ===========================
# TRAINING STEP
# ===========================
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [10]:
# ===========================
# IMAGE SAVING FUNCTION
# ===========================
def save_generated_images(epoch, seed):
    predictions = generator(seed, training=False)

    fig = plt.figure(figsize=(6, 6))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap="gray")
        plt.axis("off")

    path = os.path.join(IMAGE_DIR, f"epoch_{epoch:04d}.png")
    plt.savefig(path)
    plt.close()


In [11]:
# ===========================
# TRAINING LOOP
# ===========================
seed = tf.random.normal([16, LATENT_DIM])
history = {"gen": [], "disc": []}

print("Starting Training...")

for epoch in range(1, EPOCHS + 1):
    gen_losses = []
    disc_losses = []

    for image_batch in dataset:
        g_loss, d_loss = train_step(image_batch)
        gen_losses.append(g_loss.numpy())
        disc_losses.append(d_loss.numpy())

    history["gen"].append(np.mean(gen_losses))
    history["disc"].append(np.mean(disc_losses))

    print(f"Epoch {epoch}/{EPOCHS} | G: {history['gen'][-1]:.4f} | D: {history['disc'][-1]:.4f}")

    if epoch % SAVE_INTERVAL == 0:
        save_generated_images(epoch, seed)

        generator.save(os.path.join(CHECKPOINT_DIR, f"generator_epoch_{epoch}.keras"))
        discriminator.save(os.path.join(CHECKPOINT_DIR, f"discriminator_epoch_{epoch}.keras"))


Starting Training...
Epoch 1/150 | G: 0.6567 | D: 1.3345
Epoch 2/150 | G: 0.7273 | D: 1.3470
Epoch 3/150 | G: 0.7322 | D: 1.3531
Epoch 4/150 | G: 0.7578 | D: 1.3292
Epoch 5/150 | G: 0.7875 | D: 1.3014
Epoch 6/150 | G: 0.8244 | D: 1.2811
Epoch 7/150 | G: 0.8207 | D: 1.2777
Epoch 8/150 | G: 0.7996 | D: 1.3054
Epoch 9/150 | G: 0.7696 | D: 1.3331
Epoch 10/150 | G: 0.7830 | D: 1.3205
Epoch 11/150 | G: 0.7688 | D: 1.3328
Epoch 12/150 | G: 0.7790 | D: 1.3217
Epoch 13/150 | G: 0.7675 | D: 1.3266
Epoch 14/150 | G: 0.7706 | D: 1.3300
Epoch 15/150 | G: 0.7878 | D: 1.3144
Epoch 16/150 | G: 0.7794 | D: 1.3218
Epoch 17/150 | G: 0.7707 | D: 1.3271
Epoch 18/150 | G: 0.7946 | D: 1.3155
Epoch 19/150 | G: 0.7744 | D: 1.3256
Epoch 20/150 | G: 0.7835 | D: 1.3211
Epoch 21/150 | G: 0.7906 | D: 1.3170
Epoch 22/150 | G: 0.7724 | D: 1.3233
Epoch 23/150 | G: 0.7798 | D: 1.3238
Epoch 24/150 | G: 0.8002 | D: 1.3104
Epoch 25/150 | G: 0.7931 | D: 1.3131
Epoch 26/150 | G: 0.8017 | D: 1.3120
Epoch 27/150 | G: 0.7962 |

In [12]:
# ===========================
# SAVE FINAL OUTPUTS
# ===========================
generator.save(os.path.join(MODEL_DIR, "generator_final.keras"))
discriminator.save(os.path.join(MODEL_DIR, "discriminator_final.keras"))

plt.plot(history["gen"], label="Generator Loss")
plt.plot(history["disc"], label="Discriminator Loss")
plt.legend()
plt.savefig(os.path.join(PLOT_DIR, "loss_plot.png"))
plt.close()

print("Training Complete & Everything Saved to Drive")


Training Complete & Everything Saved to Drive
